# 1. word2vec

In [1]:
# 1.数据预处理，读取停用词、分词等
# 读取停用词
import pandas as pd
import jieba
stop_words = []
with open('./stop_words.txt', "r", encoding="UTF-8") as StopWords:
    line = StopWords.readline()
    while line:
        stop_words.append(line[:-1])
        line = StopWords.readline()
stop_words = set(stop_words)

# 读取文本，预处理，分词，去除停用词，得到词典
data=pd.read_csv('./和讯网滚动新闻-600页.csv',encoding='gbk')
# 选择前8000个新闻数据
content=data.loc[1:8000,'content']
words=[]  
for line in content:
    line.replace(' ','')
    line.replace('\r','')
    line.replace('\n','')
    wlist=[word for word in jieba.cut(line, cut_all=False) if word not in stop_words]
    words.append(wlist)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/z8/8jkfgw4s2v7227nxdpgg1bcw0000gn/T/jieba.cache
Loading model cost 0.427 seconds.
Prefix dict has been built successfully.


In [2]:
# 模型初始化
from gensim.models import Word2Vec
# skip-gram模型
model_skipgram = Word2Vec(words, vector_size=50, min_count=1, sg=1)
# CBOW模型
model_CBOW = Word2Vec(words, vector_size=50, min_count=1, sg=0)

In [3]:
# 查看数据——skip-gram模型
model_skipgram.wv['社会']
for key in model_skipgram.wv.similar_by_word("社会", topn = 10):
    print(key)

('深远', 0.7678378820419312)
('关乎', 0.7644622921943665)
('百姓生活', 0.7619006037712097)
('公益事业', 0.7605026364326477)
('日趋严重', 0.759941816329956)
('增质', 0.7588986158370972)
('有益于', 0.7549617886543274)
('重视', 0.7547312378883362)
('慈善事业', 0.7535605430603027)
('道德规范', 0.7514213919639587)


In [4]:
# 查看数据——CBOW模型
model_CBOW.wv['社会']
for key in model_CBOW.wv.similar_by_word("社会", topn = 10):
    print(key)

('民生问题', 0.7298255562782288)
('各方', 0.7283612489700317)
('倡导', 0.7146118879318237)
('意义', 0.7111020684242249)
('职业', 0.7072268724441528)
('重要途径', 0.7055273056030273)
('民生', 0.6991469860076904)
('福祉', 0.6980387568473816)
('重视', 0.69774329662323)
('国民经济', 0.6976528763771057)


# 2. BERT

In [1]:
# 导入BERT相关包
import torch
from transformers import BertTokenizer, BertModel

In [2]:
# 初始化分词器和BERT模型
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertModel.from_pretrained("bert-base-chinese")

In [3]:
# 初始化输入，这里仅作为例子
inputs = tokenizer("词嵌入非常有意思", "我很喜欢词嵌入", return_tensors = "pt")
inputs

{'input_ids': tensor([[ 101, 6404, 2316, 1057, 7478, 2382, 3300, 2692, 2590,  102, 2769, 2523,
         1599, 3614, 6404, 2316, 1057,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
# 分词、并查看输出
tokenizer.decode(inputs["input_ids"].data.cpu().numpy().reshape(-1))
outputs = model(**inputs)
outputs
# 在接下来研究者可以根据这一词嵌入继续进行其后的操作

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1357, -0.4644, -0.6541,  ...,  0.2003, -0.7489, -0.1722],
         [-0.5895,  0.6582,  0.2776,  ..., -0.8723,  0.0908, -0.1822],
         [ 0.5337, -0.1124, -0.2553,  ...,  0.9628,  0.0113, -0.2751],
         ...,
         [ 0.0826, -0.1929, -0.5683,  ...,  0.8724, -0.3365,  0.0299],
         [-0.0312,  0.7349, -0.0538,  ...,  0.7425, -0.2553, -0.0180],
         [-0.5033, -1.0125,  0.2439,  ...,  0.0756, -0.6521, -0.4169]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 1.0000, -0.9706,  1.0000, -0.9744, -0.9759, -0.9857,  0.9719,  0.9906,
         -0.3052, -1.0000,  1.0000,  1.0000,  0.9909,  0.9840,  1.0000, -1.0000,
          0.9624, -0.6737,  0.9979, -0.7521, -0.9905, -1.0000,  0.9993, -1.0000,
         -0.9922,  0.9999,  0.9985,  0.9835,  0.9706,  1.0000,  0.9974,  1.0000,
          0.9874,  0.9868, -1.0000, -0.9184,  0.9402,  0.9998,  0.9759,  0.9262,
         -1.0000,  0.9994, -0.99